In [44]:
import mip
from mip import xsum, minimize, BINARY, OptimizationStatus

In [45]:
mipmodel = mip.Model()

In [46]:
PCs = ["PC1", "PC2", "PC3"]
Services = ["g", "p", "s"]  # generic, primary, secondary

a = {"g": 680, "p": 200, "s": 200}

b = {"PC1": 160 / 680, "PC2": 200 / 680, "PC3": 300 / 680}

d = {"PC1": 200, "PC2": 500, "PC3": 1000}

c = {"p": {"PC1": 0, "PC2": 0, "PC3": 200}, "s": {"PC1": 0, "PC2": 0, "PC3": 200}}

e = {
    "g": {
        "PC1": {"PC2": 0, "PC3": 680},
        "PC2": {"PC1": 0, "PC3": 680},
        "PC3": {"PC1": 0, "PC2": 0},
    }
}

f = {"PC1": 1000, "PC2": 1000, "PC3": 1000}

In [47]:
# x[h][j] is fraction of service h on PC j
# for the generic service g, these are continouse, non-negative variables, which is the default
# for the p and s servicer, these are binary variables

x = {}
for h in Services:
    if h == "g":
        x[h] = {j: mipmodel.add_var() for j in PCs}
    else:
        x[h] = {j: mipmodel.add_var(var_type=BINARY) for j in PCs}

# w[h][i][j] is the fraction of service h moved from PC i to PC j, (only for service g)
w = {"g": {}}
for i in PCs:
    w["g"][i] = {}
    for j in PCs:
        if j != i:
            w["g"][i][j] = mipmodel.add_var()

# y[j] is 1 if PC j is part of the final config
y = {j: mipmodel.add_var(var_type=BINARY) for j in PCs}

In [48]:
# assignmentconstraint[h] states all GB associated with service h is allocated to one or some of the PCs
assignmentconstraint = {}
for h in Services:
    mipmodel += xsum(x[h][j] for j in PCs) == 1, "ass_" + h
    assignmentconstraint[h] = "ass_" + h

changeconstraint = {}
for j in PCs:
    mipmodel += (
        x["g"][j]
        + xsum(w["g"][j][i] for i in PCs if j != i)
        - xsum(w["g"][i][j] for i in PCs if j != i)
        == b[j],
        "change_" + j,
    )
    changeconstraint[j] = "change_" + j

# capacityconstraint[j] states that the amount of GB alloated to PC j does not exceed its disk size
# (or is equal to 0 if the PC is not in the final config)
capacityconstraint = {}
for j in PCs:
    mipmodel += xsum(a[h] * x[h][j] for h in Services) - d[j] * y[j] <= 0, "cap_" + j
    capacityconstraint[j] = "cap_" + j

In [49]:
# diversityconstraint[j] states that each PC can only be designated to one of "p" or "s"
diversityconstraint = {}
for j in PCs:
    mipmodel += x["p"][j] + x["s"][j] <= 1, "diversity_" + j
    diversityconstraint[j] = "diversity_" + j

In [50]:
mipmodel.objective = minimize(
    xsum(c[h][j] * x[h][j] for h in ["p", "s"] for j in PCs)
    + xsum(e["g"][i][j] * w["g"][i][j] for i in PCs for j in PCs if j != i)
    + xsum(f[j] * y[j] for j in PCs)
)

In [54]:
print("working on solutions")
mipmodel.max_gap = 0.02
status = mipmodel.optimize(max_seconds=300)
if status == OptimizationStatus.OPTIMAL:
    print("optimal solution found - cost {} ".format(mipmodel.objective_value))
elif status == OptimizationStatus.FEASIBLE:
    print(
        "sol.cost {} found, best possible: {}".format(
            mipmodel.objective_value, mipmodel.objective_bound
        )
    )
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print(
        "no feasible solution found, lower bound is: {}".format(mipmodel.objetive_bound)
    )
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    print("solution: ")
    for h in x:
        for j in PCs:
            if x[h][j].x > 1e-6:
                print("x", h, j, x[h][j].x)

    for j in y:
        if y[j].x > 1e-6:
            print("y", j, y[j].x)
    for i in PCs:
        for j in PCs:
            if i != j and w["g"][i][j].x > 1e-6:
                print("w[g]", i, j, w["g"][i][j].x)
print("finished")

working on solutions
finished
